In [1]:
!pip install llama-index-core llama-index-llms-huggingface llama-index-embeddings-huggingface nest-asyncio python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.9 MB/s eta 0:00:00


In [3]:
!pip install llama-index-llms-ollama

In [8]:
!pip install -U llama-index-readers-file

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 18.2 MB/s eta 0:00:00


In [10]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [17]:
!apt-get install tmux

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tmux is already the newest version (3.2a-4ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [18]:
!tmux new-session -d 'ollama serve'

In [19]:
!curl http://127.0.0.1:11434

Ollama is running

In [23]:
!tmux new-session -d 'ollama run deepseek-r1:1.5b'

In [24]:
!curl http://127.0.0.1:11434/api/tags

{"models":[{"name":"deepseek-r1:1.5b","model":"deepseek-r1:1.5b","modified_at":"2025-01-24T16:43:41.135962219Z","size":1117322599,"digest":"a42b25d8c10a841bd24724309898ae851466696a7d7f3a0a408b895538ccbc96","details":{"parent_model":"","format":"gguf","family":"qwen2","families":["qwen2"],"parameter_size":"1.8B","quantization_level":"Q4_K_M"}}]}

In [31]:
#test
!curl http://127.0.0.1:11434/api/generate -d '{"model": "deepseek-r1:1.5b","prompt": "What is machine learning?","stream": false}'

{"model":"deepseek-r1:1.5b","created_at":"2025-01-24T16:49:45.80919196Z","response":"\u003cthink\u003e\n\n\u003c/think\u003e\n\nMachine learning is a subset of artificial intelligence (AI) that focuses on developing algorithms and models enable computers to learn from and make decisions or predictions based on data. Unlike traditional AI, which is based on explicit rules and hard-coded programs, machine learning systems automatically improve through data and experience.\n\n### Key Aspects of Machine Learning:\n\n1. **Learning Types**:\n   - **Supervised Learning**: Uses labeled data (input + output) to train models.\n   - **Unsupervised Learning**: Finds patterns in unlabeled data.\n   - **Reinforcement Learning**: learn from interactions with an environment, receiving rewards or penalties.\n\n2. **Common Models**:\n   - **Classification**: predicts categories (e.g., spam detection).\n   - **Regression**: predicts numerical values (e.g., stock market).\n   - **Clustering**: groups simi

In [28]:
!ollama list

NAME                ID              SIZE      MODIFIED      
deepseek-r1:1.5b    a42b25d8c10a    1.1 GB    3 minutes ago    


In [32]:
#chat with your documents

In [34]:
from dotenv import load_dotenv
from IPython.display import Markdown, display

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, ServiceContext, SimpleDirectoryReader


#allows nested access to the event loop
nest_asyncio.apply()
#add documents in this directory
input_dir_path = '/content/'

# setup llm & embedding model
llm=Ollama(model="deepseek-r1:1.5b", request_timeout=120.0)
embed_model = HuggingFaceEmbedding( model_name="Snowflake/snowflake-arctic-embed-m", trust_remote_code=True)
#embed_model = HuggingFaceEmbedding( model_name="BAAI/bge-large-en-v1.5", trust_remote_code=True)


In [35]:

#load data
loader = SimpleDirectoryReader(
            input_dir = input_dir_path,
            required_exts=[".pdf"],
            recursive=True
        )
docs = loader.load_data()

#creating an index over loaded data
Settings.embed_model = embed_model
index = VectorStoreIndex.from_documents(docs, show_progress=True)

#create the query engine, where we use a cohere reranker on the fetched nodes
Settings.llm = llm
query_engine = index.as_query_engine()

# ====== customise prompt template ======
qa_prompt_tmpl_str = (
"Context information is below.\n"
"---------------------\n"
"{context_str}\n"
"---------------------\n"
"Given the context information above I want you to think step by step to answer the query in a crisp manner, incase case you don't know the answer say 'I don't know!'.\n"
"Query: {query_str}\n"
"Answer: "
)
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)


Parsing nodes:   0%|          | 0/26 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/50 [00:00<?, ?it/s]

In [37]:
!curl http://127.0.0.1:11434

Ollama is running

In [39]:
#generate the response
response = query_engine.query("What exactly are MLMs ?",)

display(Markdown(str(response)))


<think>
Okay, so I need to figure out what exactly are Multimodal Learning Models (MLMs) are. From the context provided, it seems like there's a lot of information about multimodal large language models. Let me start by looking through the sections and see if any mention multimodal learning.

Looking at the first section under "Query Details," I see a paragraph that mentions multimodal large language models being grounded to the world. That sounds important. It talks about how they're trained using multiple data sources or modalities—like text, images, etc.—to improve understanding beyond single modality. So, this might be key.

Then in another section under "Case Information," there's a reference to a paper titled "Kosmos-2: Grounding multimodal large language models to the world." That reinforces the idea that these models are combining text and other modalities for better performance. 

In the context information provided, there are several paragraphs discussing multimodal models. For example, in section 83, it mentions Gemini 1.5: Unlocking multimodal understanding across millions of tokens. This is a specific model that uses multiple languages or datasets to train effectively.

Also, under Case Information, there's a reference to Qwen-VL being a large vision-language model with versatile abilities. That again points towards combining vision (like images) and text data.

So putting it all together, multimodal learning models are designed to process information from multiple sources or formats, like text and images, rather than just relying on one type of data. This allows them to understand and answer questions more comprehensively, improving performance in tasks that require integrating different types of data.
</think>

Multimodal Learning Models (MLMs) are advanced machine learning models designed to process and integrate information from multiple sources or formats, such as text, images, audio, etc., rather than relying solely on a single type of data. These models enhance understanding by combining various modalities, leading to better performance in tasks that require integrating different types of data.